## Analyzing US Economic Data and Building a Dashboard

### Summary
Gross domestic product (GDP) is a measure of the market value of all the final goods and services produced in a period. GDP is an indicator of how well the economy is doing. A drop in GDP indicates the economy is producing less; similarly an increase in GDP suggests the economy is performing better. In this assignment you will examine how changes in GDP impact the unemployment rate.

### Contents
 - [Seeting up the environment](#section0)
 - [Create a dataframe that contains the GDP data and display using the method head()](#section1)
 - [Create a dataframe that contains the unemployment data. Display the first five rows of the dataframe using the method head()](#section2)
 - [Display a dataframe where unemployment was greater than 8.5%](#section3)
 - [Use the function make_dashboard to make a dashboard](#section4)
 - [Save the dashboard on the IBM Cloud Object Storage](#section5)

<a id="section0"></a>

### Setting up the environment

In [30]:
! conda install bokeh

import pandas as pd # For dataframe manipulation
from bokeh.io import push_notebook, output_file,show, output_notebook # To build the dashboard
from bokeh.plotting import figure
import os # For file manipulation
import boto3 # For comunication with IBM Cloud Storage API, to store the dashboard

output_notebook()


# Defining data links
gdp_url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/projects/coursera_project/clean_gdp.csv'
unemployment_url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/projects/coursera_project/clean_unemployment.csv'

# Getting dataframes
gdp_df = pd.read_csv(gdp_url)
unemployment_df = pd.read_csv(unemployment_url)

Solving environment: done

# All requested packages already installed.



Loading BokehJS ...

<a id="section1"></a>

### Create a dataframe that contains the GDP data and display using the method head()

In [37]:
gdp_df.head()

,date,level-current,level-chained,change-current,change-chained
0,1948,274.8,2020.0,-0.7,-0.6
1,1949,272.8,2008.9,10.0,8.7
2,1950,300.2,2184.0,15.7,8.0
3,1951,347.3,2360.0,5.9,4.1
4,1952,367.7,2456.1,6.0,4.7


<a id="section2"></a>

### Create a dataframe that contains the unemployment data. Display the first five rows of the dataframe using the method head()

In [38]:
unemployment_df.head()

,date,unemployment
0,1948,3.750000
1,1949,6.050000
2,1950,5.208333
3,1951,3.283333
4,1952,3.025000


<a id="section3"></a>

### Display a dataframe where unemployment was greater than 8.5%

In [39]:
unemployment_df[unemployment_df['unemployment']>8.5]

,date,unemployment
34,1982,9.708333
35,1983,9.600000
61,2009,9.283333
62,2010,9.608333
63,2011,8.933333


<a id="section4"></a>

### Use the function make_dashboard to make a dashboard

In [40]:
# Defining make_dashboard function
def make_dashboard(x, gdp_change, unemployment, title, file_name):
    output_file(file_name)
    output_notebook()
    p = figure(title=title, x_axis_label='Year', y_axis_label='Percentage')
    p.line(x.squeeze(), gdp_change.squeeze(), color="firebrick", line_width=4, legend="% GDP change")
    p.line(x.squeeze(), unemployment.squeeze(), line_width=4, legend="% unemployed")
    show(p)

In [41]:
# Call the funciton to build the dashboard
make_dashboard(gdp_df[["date"]], gdp_df[["change-current"]], unemployment_df[["unemployment"]], "Corelation between unemployment and GDP", "index.html")

Loading BokehJS ...

<a id="section5"></a>

### Save the dashboard on the IBM Cloud Object Storage

In [42]:
## Setting IBM Cloud API credentials
# Setting endpoint
end_point = "https://s3.eu-de.cloud-object-storage.appdomain.cloud"
bucket_name = "joaos3ibm"

# setting credentials
credentials = {
  "apikey": "zlH3BKP2PDzzH4gctlrUZpBUsAtBEBTIkaN5RRY0bFjW",
  "cos_hmac_keys": {
    "access_key_id": "349d8b26f42d4386b7612b8bccb71ae8",
    "secret_access_key": "3d251e2740d17841fd7466f378df74738cf6d1ce81cf4340"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto-generated for key 349d8b26-f42d-4386-b761-2b8bccb71ae8",
  "iam_apikey_name": "credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Manager",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/c6a9438ea1614f9bb54bcb9b9024a630::serviceid:ServiceId-594bf6bf-7398-405d-a50e-1722e93c2f97",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/c6a9438ea1614f9bb54bcb9b9024a630:b01de0a2-b6d9-49b6-9b32-93a99a447dda::"
}

file_name = 'index.html'

# Getting the file
file_path = os.getcwd() + "/" + file_name
file = open(file_path, "r")


# Creating a API resource
resource = boto3.resource(
    's3',
    aws_access_key_id = credentials["cos_hmac_keys"]['access_key_id'],
    aws_secret_access_key = credentials["cos_hmac_keys"]["secret_access_key"],
    endpoint_url = end_point,
)

resource.Bucket(name=bucket_name).put_object(Key=file_name, Body=file.read())

# Creating a API client to retrive the URL of object created
client = boto3.client(
    's3',
    aws_access_key_id = credentials["cos_hmac_keys"]['access_key_id'],
    aws_secret_access_key = credentials["cos_hmac_keys"]["secret_access_key"],
    endpoint_url=end_point,

)

# Get the url for the object created
url = client.generate_presigned_url('get_object', Params={'Bucket': bucket_name,'Key': file_name}, ExpiresIn=500000)
print(url)


https://s3.eu-de.cloud-object-storage.appdomain.cloud/joaos3ibm/index.html?Signature=kCI%2FT1PZYOQU3kgNfyeyKZ6%2BSW0%3D&AWSAccessKeyId=349d8b26f42d4386b7612b8bccb71ae8&Expires=1559192537
